In [1]:
import pandas , numpy
import psycopg2
from dotenv import load_dotenv
import os
import time

In [2]:
load_dotenv()

True

In [3]:
#CONNECTING TO COMPANY DB
db_name = "OpenAlex"
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")

In [4]:
conn = psycopg2.connect(f"dbname ={db_name} user={user} password={password} host={host} port={port}")

In [5]:
#getting top 5 papers for each field
#bronchitis
query = """
WITH sarcoma_related_mesh AS (
    SELECT DISTINCT work_id
    FROM works_mesh wm
    WHERE wm.descriptor_name ILIKE '%bronchitis%'
)
SELECT
    w.id AS work_id,
    w.title,
    w.cited_by_count
FROM works w
JOIN sarcoma_related_mesh srm ON w.id = srm.work_id
WHERE w.cited_by_count IS NOT NULL
ORDER BY
    w.cited_by_count DESC
LIMIT 5;
"""

In [6]:
# cursor = conn.cursor()
# cursor.execute(query)
# #cursor.execute(query , (["%cancer%"],))
# rows = cursor.fetchall()
# print(rows)

In [ ]:
dict_diseases = {
'Diabetes': ['%diabetes%', '%hyperglycemia%'],
 'Lymphoma': ['%lymphoma%', '%Hodgkin%', '%lymphatic%'],
 'Sarcoma': ['%sarcoma%'],
'Cancer': ['%cancer%', '%carcinoma%', '%neoplasm%', '%tumor%', '%malignant%']}

In [8]:
def number_authors_query(keyword = None, lst_keywords = None):
    if keyword:
        query = f"""
            WITH sarcoma_related_mesh AS (
                SELECT DISTINCT work_id
                FROM works_mesh wm
                WHERE wm.descriptor_name ILIKE '{keyword}' 
            )
            SELECT
                COUNT(DISTINCT wa.author_id) AS unique_author_count
            FROM works w
            JOIN sarcoma_related_mesh srm ON w.id = srm.work_id
            JOIN works_authorships wa ON w.id = wa.work_id;
            """
    else:
        formatted_keywords = ", ".join([f"'{kw}'" for kw in lst_keywords])
        query = f"""
            WITH sarcoma_related_mesh AS (
                SELECT DISTINCT work_id
                FROM works_mesh wm
                WHERE wm.descriptor_name ILIKE ANY(ARRAY[{formatted_keywords}])
            )
            SELECT
                COUNT(DISTINCT wa.author_id) AS unique_author_count
            FROM works w
            JOIN sarcoma_related_mesh srm ON w.id = srm.work_id
            JOIN works_authorships wa ON w.id = wa.work_id;
            """
    return query

In [ ]:
#running all the diseases
for key , value in dict_diseases.items():
    print(f"running {key}")
    before_running = time.time()
    if len(value) == 1:
        query = number_authors_query(keyword = value[0])
    else:
        query = number_authors_query(lst_keywords = value)

    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    after_running = time.time()
    print(rows)
    print(f"Ran in {after_running - before_running} seconds")
#ready
# True
# running Bronchitis
# [(53937,)]
# Ran in 819.9056537151337 seconds
# running Covid
# [(1075978,)]
# Ran in 1213.0744411945343 seconds
# running Alzheimer's
# [(469147,)]
# Ran in 1143.001962184906 seconds
# running Diabetes

running Lymphoma


In [ ]:
# query = """
#         WITH sarcoma_related_mesh AS (
#             SELECT DISTINCT work_id
#             FROM works_mesh wm
#             WHERE wm.descriptor_name ILIKE '%sarcoma%'
#         )
#         SELECT
#             COUNT(DISTINCT wa.author_id) AS unique_author_count
#         FROM works w
#         JOIN sarcoma_related_mesh srm ON w.id = srm.work_id
#         JOIN works_authorships wa ON w.id = wa.work_id;
#         """

In [ ]:
# cursor = conn.cursor()
# cursor.execute(query)
# rows = cursor.fetchall()
# print(rows)
# #394603